![Header.jpeg](banner_sr.jpg)

# Subtitle Generator Project Using Speech Recognition

---

#### Import Library

In [16]:
import speech_recognition as sr
from pydub import AudioSegment, silence
import srt
import datetime

#### Import Audio

In [17]:
audio_file_path = "audio_source_3.wav"
audio_segment = AudioSegment.from_wav(audio_file_path)

#### Audio Segmentation

In [ ]:
voice_segments = silence.detect_nonsilent(audio_segment, min_silence_len=300, silence_thresh=-40)
voice_segments = [(int(start / 1000), int(end / 1000)) for start, end in voice_segments]

In [19]:
merged_segments = []
skip_indices = set()

In [20]:
for i, (start, end) in enumerate(voice_segments):
    if i in skip_indices:
        continue

    duration = end - start
    if duration < 3 and i + 1 < len(voice_segments):
        next_end = voice_segments[i + 1][1]
        merged_segments.append((start, next_end))
        skip_indices.add(i + 1)
    else:
        merged_segments.append((start, end))

In [21]:
recognizer = sr.Recognizer()
subtitles = []

In [ ]:
with sr.AudioFile(audio_file_path) as audio_source:
    for idx, (start_sec, end_sec) in enumerate(merged_segments):
        duration = end_sec - start_sec
        audio_source_offset = start_sec
        try:
            audio_data = recognizer.record(audio_source, duration=duration)
            recognized_text = recognizer.recognize_google(audio_data, language="id-ID")

            subtitle = srt.Subtitle(
                index=idx + 1,
                start=datetime.timedelta(seconds=start_sec),
                end=datetime.timedelta(seconds=end_sec),
                content=recognized_text
            )
            subtitles.append(subtitle)
            print(f"Subtitle Line {idx + 1}: {recognized_text}")
        except sr.UnknownValueError:
            print(f"Subtitle {idx + 1} ...")
        except Exception as e:
            print(f"Segment Error {idx + 1}: {e}")

Subtitle 1...
Subtitle 2 berhasil: Halo teman-teman selamat datang di mata kuliah praktikum bulan
Subtitle 3 berhasil: universitas
Subtitle 4 berhasil: pada pertemuan ini
Subtitle 5 berhasil: akan membahas tentang analisis univariat bivariat dan juga
Subtitle 6...
Subtitle 7 berhasil: mari kita mulai dengan analisis univariat
Subtitle 8 berhasil: Analisis univariat adalah analisis
Subtitle 9 berhasil: yang melibatkan hanya satu variabel
Subtitle 10 berhasil: tujuan dari analisis ini adalah untuk mendefinisikan
Subtitle 11 berhasil: ringkasnya
Subtitle 12 berhasil: dan menganalisis pola yang ada di dalamnya beberapa pola yang bisa diidentifikasi dari analisis ini diantaranya
Subtitle 13 berhasil: adalah rata-rata modus median
Subtitle 14 berhasil: varians kuartil dan juga simpangan
Subtitle 15 berhasil: analisis UNY
Subtitle 16 berhasil: ini juga bisa digunakan untuk dua tipe variabel pertama variabel numerik yang grafiknya dapat berupa histo
Subtitle 17 berhasil: dan boxplot
Subtitle 1

##### Write Subtitle

In [23]:
with open("output_subtitle.srt", "w", encoding="utf-8") as subtitle_file:
    subtitle_file.write(srt.compose(subtitles))